In [1]:
import pandas as pd
import pickle

endgame = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/tic-tac-toe/tic-tac-toe.data',header=None)

def charToMove(x):
    if (x=='x'): return 1.0 
    elif (x=='o'): return -1.0 
    else: return 0.0
    
def labelToFeature(x):
    if (x=='positive'): return 1.0 
    elif (x=='negative'): return 0.0 
    
endgame_matrix = endgame.apply(lambda x:  ([charToMove(x[0]),charToMove(x[3]),charToMove(x[6]),
                         charToMove(x[1]),charToMove(x[4]),charToMove(x[7]),
                         charToMove(x[2]),charToMove(x[5]),charToMove(x[8])],
                          labelToFeature(x[9])),axis=1).as_matrix()

In [2]:
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib
from IPython.display import display
from ipywidgets import widgets, Button, HBox, VBox
import tensorflow.contrib.learn as learn
import tensorflow as tf
import numpy as np
import random
#sess = tf.Session()
#init = tf.global_variables_initializer()

def cnn_model_fn(features_cnn, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features_cnn, [-1, 3, 3, 1])

  #  Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=4,
      kernel_size=[2, 2],
      padding="same",
      activation=tf.nn.relu)

  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[1, 1], strides=1)

  # Dense Layer
  pool2_flat = tf.reshape(pool1, [-1, 3 * 3 * 4])
  #print(sess.run(pool2_flat))
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(inputs=dense, rate=0.4, training=mode == learn.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=2)

  loss = None
  train_op = None

  # Calculate Loss (for both TRAIN and EVAL modes)
  if mode != learn.ModeKeys.INFER:
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=2)
    loss = tf.losses.softmax_cross_entropy(
        onehot_labels=onehot_labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == learn.ModeKeys.TRAIN:
    train_op = tf.contrib.layers.optimize_loss(
        loss=loss,
        global_step=tf.contrib.framework.get_global_step(),
        learning_rate=0.001,
        optimizer="SGD")

  # Generate Predictions
  predictions = {
      "classes": tf.argmax(
          input=logits, axis=1),
      "probabilities": tf.nn.softmax(
          logits, name="softmax_tensor")
  }
  predictions = tf.nn.softmax(
          logits, name="softmax_tensor")

  # Return a ModelFnOps object
  return model_fn_lib.ModelFnOps(
      mode=mode, predictions=predictions, loss=loss, train_op=train_op)

def cnn_predict(model,data):
    p = model.predict(x=data, as_iterable=True)
    pred=[]
    for x in p:
        pred.append(x)
    return(np.array(pred))

def ffnn_predict(model,data):
    return(np.array(list(model.predict_proba(input_fn=new_data))))



def train_cnn(data,steps=200):
    # Create the Estimator
    cnn_classifier = learn.Estimator(model_fn=cnn_model_fn)

    features_cnn = []
    labels_cnn = []

    for move in data:
        asFloat = move[1]#.astype(float)
        features_cnn.append(move[0][0:9])
        labels_cnn.append(asFloat)
    features_cnn = np.asarray(features_cnn,dtype=np.float32)
    labels_cnn = np.asarray(labels_cnn,dtype=np.int32)
    cnn_classifier.fit(
      x=features_cnn,
      y=labels_cnn,
      batch_size=10000,
      steps=steps)
    return(cnn_classifier)

def train(data,steps=200):
    features = []
    response = []
    for play in data:
        features.append(play[0])
        #if (play[1]): r=1.0 
        #else: r= 0.0
        response.append(play[1])
        
    features = np.array(features,dtype=float)
    response = np.array(response,dtype=float)

    # Specify that all features have real-value data
    feature_columns = [tf.contrib.layers.real_valued_column("", dimension=9)]
    classifier  = learn.DNNClassifier(
        feature_columns=feature_columns,
        n_classes=2,
        hidden_units=[20])

    # Fit model.
    classifier.fit(x=features,
                   y=response,
                   steps=steps)
    return(classifier)

model_pre = train(endgame_matrix,500)
model_cnn_pre = train_cnn(endgame_matrix,2500)

f = open('moves.pckl', 'rb')
moves = pickle.load(f)
f.close()


model_ffnn_laygo = train(moves,500)
model_cnn_laygo = train_cnn(random.sample(moves,200),2500)

model_cnn_laygo_hist = model_cnn_laygo
model_ffnn_laygo_hist = model_ffnn_laygo

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x106b4c128>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': None}
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving int

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/util/deprecation.py:248: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  equality = a == b


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/rm/dr73r_t919b9ftbj_hgqzc51qmbg7w/T/tmplfr72bmx/model.ckpt.
INFO:tensorflow:loss = 0.755893, step = 1
INFO:tensorflow:global_step/sec: 384.255
INFO:tensorflow:loss = 0.507136, step = 101 (0.262 sec)
INFO:tensorflow:global_step/sec: 354.1
INFO:tensorflow:loss = 0.399236, step = 201 (0.282 sec)
INFO:tensorflow:global_step/sec: 376.243
INFO:tensorflow:loss = 0.255461, step = 301 (0.266 sec)
INFO:tensorflow:global_step/sec: 364.1
INFO:tensorflow:loss = 0.159487, step = 401 (0.277 sec)
INFO:tensorflow:Saving checkpoints for 500 into /var/folders/rm/dr73r_t919b9ftbj_hgqzc51qmbg7w/T/tmplfr72bmx/model.ckpt.
INFO:tensorflow:Loss for final step: 0.112559.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11206ef60>, '_master': '', '_num_ps_replicas': 0, '_num_wor

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/rm/dr73r_t919b9ftbj_hgqzc51qmbg7w/T/tmpz8oqb91v/model.ckpt.
INFO:tensorflow:loss = 0.664931, step = 1
INFO:tensorflow:global_step/sec: 441.26
INFO:tensorflow:loss = 0.40119, step = 101 (0.227 sec)
INFO:tensorflow:global_step/sec: 425.25
INFO:tensorflow:loss = 0.388204, step = 201 (0.235 sec)
INFO:tensorflow:global_step/sec: 465.673
INFO:tensorflow:loss = 0.381711, step = 301 (0.215 sec)
INFO:tensorflow:global_step/sec: 438.721
INFO:tensorflow:loss = 0.376849, step = 401 (0.228 sec)
INFO:tensorflow:Saving checkpoints for 500 into /var/folders/rm/dr73r_t919b9ftbj_hgqzc51qmbg7w/T/tmpz8oqb91v/model.ckpt.
INFO:tensorflow:Loss for final step: 0.372371.
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x112562ba8>, '_master': '', '_num_ps_replicas': 0, '_num_wo

In [6]:

tf.logging.set_verbosity(tf.logging.ERROR)


#moves = []
#s = np.array([])
o_starts = False


def new_data():
    possible_moves = []
    for play in calculate_plays(board_to_matrix().flatten()):
        possible_moves.append(board_to_numeric(play[0]))
    return np.array(possible_moves,dtype=np.float32)*-1


def reset_board():
    for item in items:
        item.description = '-'
        item.disabled=False

def board_to_matrix():
    return(np.array([[items[0].description, items[3].description, items[6].description], [items[1].description, items[4].description, items[7].description], [items[2].description, items[5].description, items[8].description]]).transpose())

        
def score_board():
    for i in ['x','o']:
        vertical = ((items[0].description == i) & (items[1].description == i) & (items[2].description == i)) | ((items[3].description == i) & (items[4].description == i) & (items[5].description == i)) | ((items[6].description == i) & (items[7].description == i) & (items[8].description == i))
        horizontal = ((items[0].description == i) & (items[3].description == i) & (items[6].description == i)) | ((items[1].description == i) & (items[4].description == i) & (items[7].description == i)) | ((items[2].description == i) & (items[5].description == i) & (items[8].description == i))
        diagonal = ((items[0].description == i) & (items[4].description == i) & (items[8].description == i)) | ((items[2].description == i) & (items[4].description == i) & (items[6].description == i))
        if (vertical | horizontal | diagonal):
            finished=True
            winner=i
            return(finished,winner)
    return(False,'')
    
def calculate_plays(b):
    plays = []
    for i in range(0, b.size):
        if (b[i]=='-'):
            r=b.tolist()
            r[i] = 'x'
            plays.append((r,i))
    return(plays)


def board_to_numeric(board):
    board_num = []
    for element in board:
        if (element =='x'): val = 1.0
        elif (element == 'o'): val = -1.0
        else: val = 0.0
        board_num.append(val)
    return(board_num)




def play_on_click(sender):
    global o_starts, model_ffnn_laygo, model_cnn_laygo

    before = board_to_matrix().flatten()
    plays = calculate_plays(before)
    if (sender.description=='-'):
        sender.description='x'
        sender.disabled= True
        finished,winner = score_board()
        if (finished):
            o_starts = not o_starts
            if (o_starts):
                if (strategy.value=='FFNN - LAYGO'):
                    model_ffnn_laygo= train(random.sample(moves,200),500)
                elif (strategy.value=='CNN - LAYGO'):
                    model_cnn_laygo= train_cnn(random.sample(moves,200),500)
            reset_board()
            if (o_starts):
                play_o()
            return()
        after = board_to_matrix().flatten()
        plays_with_tf = []
        for play in plays:
            value = (board_to_numeric(play[0]),(play[0]==after).sum() == len(play[0]),play[1])
            plays_with_tf.append(value)
            moves.append(value)
        finished,winner = score_board()
        if (finished):
            reset_board()
            o_starts = not o_starts
            if (o_starts):
                play_o()
        else:
            play_o()


def play_o():
    if (strategy.value=='FFNN - LAYGO'):
        model = model_ffnn_laygo # train(moves,500)
        predict = ffnn_predict
    elif (strategy.value=='CNN - LAYGO'):
        model = model_cnn_laygo # train_cnn(moves,500)
        predict = cnn_predict
    elif (strategy.value=='CNN - LAYGO - HIST'):
        model = model_cnn_laygo_hist # train_cnn(moves,500)
        predict = cnn_predict
    elif (strategy.value=='FFNN - LAYGO - HIST'):
        model = model_ffnn_laygo_hist # train_cnn(moves,500)
        predict = ffnn_predict
    elif (strategy.value=='CNN'):
        model = model_cnn_pre
        predict = cnn_predict
    elif (strategy.value=='FFNN'):
        model = model_pre
        predict = ffnn_predict

    # let's calculate the probabilities of the possible moves
    possible_moves = calculate_plays(board_to_matrix().flatten())
    predictions = predict(model,new_data())
    move_index = predictions[:,1].argmax()
    move = possible_moves[move_index][1]
    items[move].description ='o'


    
words = ['-', '-', '-', '-', '-', '-', '-', '-','-']
items = [Button(description=w) for w in words]
for item in items:
    item.on_click(play_on_click)


def new_game_on_click(sender):
    for item in items:
        item.disabled= False
        item.description='-'


strategy = widgets.Dropdown(
    options=['FFNN - LAYGO', 'CNN - LAYGO','FFNN - LAYGO - HIST', 'CNN - LAYGO - HIST', 'FFNN', 'CNN'],
    value='FFNN - LAYGO',
    description='Strategy:',
    disabled=False,
    button_style='' # 'success', 'info', 'warning', 'danger' or ''
)

new_game = widgets.Button(description="New Game")
new_game.on_click(new_game_on_click)

HBox([strategy,VBox([items[0], items[1], items[2]]), VBox([items[3], items[4], items[5],new_game]), VBox([items[6], items[7], items[8]])])



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/util/deprecation.py:248: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  equality = a == b


In [4]:
f = open('moves.pckl', 'wb')
pickle.dump(moves, f)
f.close()

In [5]:
# model_cnn_laygo = train_cnn(moves,5000)